## Plane-DEM intersections

To test the routine for calculating the plane-DEM intersection, a few simulated topographic surfaces were used.

### Test case 1

The first test case is illustrated in the image below. We have a horizontal topographic surface, at a height of 0, with 100 x 100 cells with a cell size of 1. The geological plane dips 45° towards East. The source point for the plane is located at (0, 50, 50).

The locations of the expected intersection points are (50, *, 0).

![Test case 1](ims/inters_dem_plane_case_1.png)

First, a horizontal plane was created with Saga GIS and saved in *pygsf/example_data/horiz_plane.asc*.

Loading the dataset can be made with the following function:

In [4]:
from geoprocess.libs_utils.gdal.gdal import try_read_raster_band

In [5]:
source_data = "/home/mauro/Documents/projects/gsf/pygsf/example_data/horiz_plane.asc"

In [6]:
success, cntnt = try_read_raster_band(raster_source=source_data)

In [7]:
print(success)

False


We read the data source with success. So we may unpack the result.

In [8]:
geotransform, projection, band_params, data = cntnt

ValueError: too many values to unpack (expected 4)

In [ ]:
type(geotransform)

In [ ]:
print(geotransform)

In [ ]:
type(projection)

In [ ]:
print(projection)

Hmmm, there is no projection info. In fact, there shouldn't..

In [ ]:
type(band_params)

A dictionary, as suspected. Try to see the content..

In [ ]:
print(band_params)

A very horizontal surface, we agree..

In [ ]:
type(data)

In [ ]:
data.shape

In [ ]:
data.min()

In [ ]:
data.max()

Given these data, we store them into a GeoArray, a class imported from pygsf.spatial.rasters.geoarray:

In [ ]:
from pygsf.spatial.rasters.geoarray import GeoArray

In [ ]:
ga = GeoArray(inGeotransform=geotransform, inProjection=projection, inLevels=[data])

There is a single band provided in the geoarray, and represented by the data array.

The signature of the plane-DEM intersection function is:

**plane_dem_intersection** *(srcPlaneAttitude: Plane, srcPt: Point, geo_array: GeoArray, level_ndx: int=0) 
-> Tuple[List[Point], List[Point]]:*

We already have the geoarray, we need to define the source plane attitue and the source point.

The geoplane is East-dipping with a dip angle of 45°:

In [ ]:
from pygsf.orientations.orientations import Plane
gplane = Plane(azim=90.0, dip_ang=45.0)

In [ ]:
print(gplane)

The source point is located at (0, 50, 50)

In [ ]:
from pygsf.spatial.vectorial.vectorial import Point
pt = Point(0, 50, 50)

Now we try calculating the intersection:

In [ ]:
from pygsf.topography.plane_intersect import plane_dem_intersection
inters_pts = plane_dem_intersection(
        srcPlaneAttitude=gplane,
        srcPt=pt,
        geo_array=ga)

In [ ]:
print(inters_pts)

As expected, all the intersection points lie at (50, *, 0)

Plotting with Bokeh..

In [ ]:
from bokeh.plotting import figure, output_notebook, show
x = list(map(lambda pt: pt.x, inters_pts))
y = list(map(lambda pt: pt.y, inters_pts))

output_notebook()
p = figure()
p.circle(x, y, size=2, color="navy", alpha=0.5)
show(p)

### Test case 2

Now we consider a horizontal plane at z = 0 as topographic surface (same as case 1) and another horizontal surface at z = 1 as geological plane. We should get no intersection.

In [ ]:
from pygsf.libs_utils.gdal.gdal import try_read_raster_band
source_data = "/home/mauro/Documents/projects/gsf/pygsf/example_data/horiz_plane.asc"
success, cntnt = try_read_raster_band(raster_source=source_data)

In [ ]:
print(success)

In [ ]:
geotransform, projection, band_params, data = cntnt
from pygsf.spatial.rasters.geoarray import GeoArray
ga = GeoArray(inGeotransform=geotransform, inProjection=projection, inLevels=[data])

The horizontal geological plane definition:

In [ ]:
from pygsf.orientations.orientations import Plane
gplane = Plane(azim=90.0, dip_ang=0.0)

The source point located at (0, 50, 1)

In [ ]:
from pygsf.spatial.vectorial.vectorial import Point
pt = Point(0, 50, 1)

In [ ]:
from pygsf.topography.plane_intersect import plane_dem_intersection
inters_pts = plane_dem_intersection(
        srcPlaneAttitude=gplane,
        srcPt=pt,
        geo_array=ga)

In [ ]:
print(inters_pts)

Ok, list is empty, as expected.

### Test case 3

Now we consider a horizontal plane at z = 0 as topographic surface (same as case 1) and another horizontal surface at z = 0 as geological plane. We should get all grid points as intersections.

The variables are the same as Case 2, apart from the point definition:

In [ ]:
pt = Point(0, 50, 0)

In [ ]:
inters_pts = plane_dem_intersection(
        srcPlaneAttitude=gplane,
        srcPt=pt,
        geo_array=ga)

In [ ]:
print(inters_pts)

They seem correct, just quite numerous..
We plot them with Bokeh.

In [ ]:
from bokeh.plotting import figure, output_notebook, show
x = list(map(lambda pt: pt.x, inters_pts))
y = list(map(lambda pt: pt.y, inters_pts))

output_notebook()
p = figure()
p.circle(x, y, size=2, color="navy", alpha=0.5)
show(p)

Looking at the Bokeh plot, we see that all cell centers have been plotted, as expected, since the topographic and the geological planes are coincident. So this is a "degenerate" case in which the intersection geometry is planar, not linear. 

###### Doc version: 2018-12-16